### ADK (Agent Development Kit)

**ADK** es el framework de Google para crear agentes de IA, pensado para arquitecturas multiagente complejas y flujos de trabajo orquestados. Permite definir agentes que se comunican, delegan tareas y usan herramientas, con integración directa en el ecosistema Google (Gemini, Vertex AI), pero es agnóstico en cuanto a modelos y despliegue. Es una alternativa a LangChain, OpenAI SDK Agents y otros frameworks de desarrollo de agentes.

### Hello World ADK

Este un primer ejemplo básico de un agente que saluda al usuario.

```python

In [2]:
import os
os.makedirs("26_ADK/1-basic-agent/greeting_agent", exist_ok=True)

Es muy importante que el fichero se llame `agent.py` y que esté en un directorio que coincida con la propiedad `name` del agente. En este caso `greeting_agent`.

In [3]:
%%writefile "26_ADK/1-basic-agent/greeting_agent/agent.py"
from google.adk.agents import Agent

root_agent = Agent(
    name="greeting_agent",
    # https://ai.google.dev/gemini-api/docs/models
    model="gemini-2.0-flash",
    description="Agente de saludo",
    instruction="""
    Eres un asistente útil que saluda al usuario. 
    Pregunta el nombre del usuario y salúdalo por su nombre.
    """,
)

Overwriting 26_ADK/1-basic-agent/greeting_agent/agent.py


Para probar, situarse en el directorio `26_ADK/1-basic-agent` y desde allí:

```bash
uv run adk web
```

Esto abrirá una interfaz web en `http://localhost:8080` donde se puede interactuar con el agente.

Con este comando, se puede ejecutar el agente en línea de comandos:

```bash
uv run adk run  greeting_agent
```

Para que funcione, es necesario tener un fichero `__init__.py` en el directorio del agente (`greeting_agent`), con el contenido:

```python
from .agent import root_agent

__all__ = ["root_agent"]
```

También, se puede ejecutar en modo API con el comando:

```bash
uv run adk api_server
```

Antes de comunicarse con el agente, hay que iniciar sesión.

```bash
curl -X POST http://localhost:8000/apps/greeting_agent/users/u_123/sessions/s_123 \
-H "Content-Type: application/json"
```

Una vez iniciada la sesión, se puede enviar un mensaje al agente:

```bash
curl -X POST http://localhost:8000/run \
-H "Content-Type: application/json" \
-d '{
  "appName": "greeting_agent",
  "userId": "u_123",
  "sessionId": "s_123",
  "newMessage": {
    "role": "user",
    "parts": [{
      "text": "Hola"
    }]
  }
}'
```

### Agente con tools

ADK ofrece flexibilidad al soportar varios tipos de tools:

- **Function Tools:** Tools personalizadas que tú creas para necesidades específicas.
- **Functions/Methods:** Funciones o métodos estándar síncronos definidos en tu código.
- **Agents-as-Tools:** Usa otro agente, posiblemente especializado, como tool dentro de un agente principal.
- **Long Running Function Tools:** Para operaciones asíncronas o que requieren mucho tiempo.
- **Built-in Tools:** Tools listas para usar para tareas comunes (ej: Google Search, Code Execution, RAG).
- **Third-Party Tools:** Tools de bibliotecas externas (ej: LangChain, CrewAI) que se integran fácilmente.


En este ejemplo, vamos a usar la tool `GoogleSearch` para buscar información en la web. Un agente solo puede tener un `built-in tool`.


In [4]:
import os
os.makedirs("26_ADK/2-tool-agent/tool_agent", exist_ok=True)

In [5]:
%%writefile "26_ADK/2-tool-agent/tool_agent/agent.py"
from google.adk.agents import Agent
from google.adk.tools import google_search

# def get_current_time() -> dict:
#     """
#     Obtiene la hora actual en formato AAAA-MM-DD HH:MM:SS
#     """
#     return {
#         "current_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#     }

root_agent = Agent(
    name="tool_agent",
    model="gemini-2.0-flash",
    description="Agente de herramientas",
    instruction="""
    Eres un asistente útil que puede usar las siguientes herramientas:
    - google_search
    """,
    tools=[google_search],
    # tools=[get_current_time],
    # tools=[google_search, get_current_time], # <--- No funciona
)


Writing 26_ADK/2-tool-agent/tool_agent/agent.py


Se puede usar una función de Python como una tool. En ese caso es muy importante que el `docstring` de la función esté bien definido, ya que ADK lo usará para decidir si usará esa tool. Además, es conveniente devolver un `dict` con el resultado de la función, ya que, si n se hace, ADK creará uno con clave  `result`. Si la función de la tool recibe parámetros, nunca poner valores por defecto ya que ADK no los soporta en este momento.

In [11]:
%%writefile "26_ADK/2-tool-agent/tool_agent/agent.py"
from google.adk.agents import Agent
from google.adk.tools import google_search
from datetime import datetime

def get_current_time() -> dict:
    """
    Obtiene la hora actual en formato AAAA-MM-DD HH:MM:SS
    """
    return {
        "current_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    }

root_agent = Agent(
    name="tool_agent",
    model="gemini-2.0-flash",
    description="Agente de herramientas",
    instruction="""
    Eres un asistente útil que puede usar las siguientes herramientas:
    - get_current_time
    """,
    tools=[get_current_time],
)


Overwriting 26_ADK/2-tool-agent/tool_agent/agent.py


### Agente con otros proveedores de modelos

ADK permite crear agentes con modelos que no son de Google usando LiteLLM.

In [13]:
import os
os.makedirs("26_ADK/3-litellm-agent/dad_joke_agent", exist_ok=True)

In [16]:
%%writefile "26_ADK/3-litellm-agent/dad_joke_agent/agent.py"
import os
import random

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm

# https://docs.litellm.ai/docs/providers/cohere
model = LiteLlm(
    model="cohere/command-a-03-2025",
    api_key=os.getenv("COHERE_API_KEY")
)

def get_dad_joke():
    """Obtiene un chiste de papá aleatorio.

    Returns:
        str: Un chiste de papá.
    """
    jokes = [
        "¿Por qué el pollo cruzó la carretera? ¡Para llegar al otro lado!",
        "¿Cómo llamas a un cinturón hecho de relojes? Una pérdida de tiempo.",
        "¿Cómo llamas a los espaguetis falsos? ¡Un impasta!",
        "¿Por qué el espantapájaros ganó un premio? ¡Porque era sobresaliente en su campo!",
    ]
    return random.choice(jokes)


root_agent = Agent(
    name="dad_joke_agent",
    model=model,
    description="Agente de chistes de papá",
    instruction="""
    Eres un asistente útil que puede contar chistes de papá. 
    Solo usa la herramienta `get_dad_joke` para contar chistes.
    """,
    tools=[get_dad_joke],
)


Overwriting 26_ADK/3-litellm-agent/dad_joke_agent/agent.py


### Structured Outputs

In [1]:
import os
os.makedirs("26_ADK/4-structured-outputs/email_agent", exist_ok=True)

In [ ]:
%%writefile "26_ADK/4-structured-outputs/email_agent/agent.py"
from google.adk.agents import LlmAgent
from pydantic import BaseModel, Field


# --- Define Output Schema ---
class EmailContent(BaseModel):
    subject: str = Field(
        description="La línea de asunto del correo electrónico. Debe ser concisa y descriptiva."
    )
    body: str = Field(
        description="El contenido principal del correo electrónico. Debe estar bien formateado con un saludo adecuado, párrafos y firma."
    )


# --- Create Email Generator Agent ---
root_agent = LlmAgent(
    name="email_agent",
    model="gemini-2.0-flash",
    instruction="""
        Eres un Asistente de Generación de Correos Electrónicos.
        Tu tarea es generar un correo electrónico profesional basado en la solicitud del usuario.

        DIRECTRICES:
        - Crea una línea de asunto apropiada (concisa y relevante)
        - Escribe un cuerpo de correo electrónico bien estructurado con:
            * Saludo profesional
            * Contenido principal claro y conciso
            * Cierre apropiado
            * Tu nombre como firma
        - Sugiere archivos adjuntos relevantes si aplica (lista vacía si no se necesitan)
        - El tono del correo electrónico debe coincidir con el propósito (formal para negocios, amigable para colegas)
        - Mantén los correos electrónicos concisos pero completos

        IMPORTANTE: Tu respuesta DEBE ser un JSON válido que coincida con esta estructura:
        {
            "subject": "Línea de asunto aquí",
            "body": "Cuerpo del correo electrónico aquí con párrafos y formato adecuados",
        }

        NO incluyas explicaciones ni texto adicional fuera de la respuesta JSON.
    """,
    description="Genera correos electrónicos profesionales con asunto y cuerpo estructurados",
    output_schema=EmailContent, # también existe una propiedad input_schema para definir la entrada esperada
    output_key="email",
)


Writing 26_ADK/4-structured-outputs/email_agent/agent.py


### Sessions and State

In [3]:
import os
os.makedirs("26_ADK/5-sessions-and-state/question_answering_agent", exist_ok=True)

El agente no tiene nada de particular. Únicamente, en la descripción se definen dos parámetros que se pasaran desde el estado de la sesión.

In [4]:
%%writefile "26_ADK/5-sessions-and-state/question_answering_agent/agent.py"
from google.adk.agents import Agent

# Create the root agent
question_answering_agent = Agent(
    name="question_answering_agent",
    model="gemini-2.0-flash",
    description="Agente de respuesta a preguntas",
    instruction="""
    Eres un asistente útil que responde preguntas sobre las preferencias del usuario.

    Aquí tienes información sobre el usuario:
    Nombre: 
    {user_name}
    Preferencias: 
    {user_preferences}
    """,
)


Writing 26_ADK/5-sessions-and-state/question_answering_agent/agent.py


La sesión se compone de un identificador único, un nombre de aplicación, un identificador de usuario y un estado que almacena información sobre el usuario y sus preferencias. El runner asocia el agente a la sesión.

In [8]:
%%writefile "26_ADK/5-sessions-and-state/basic_stateful_session.py"
import uuid

from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from question_answering_agent import question_answering_agent

load_dotenv()

async def main():
    # Create a new session service to store state
    session_service_stateful = InMemorySessionService()

    initial_state = {
        "user_name": "Brandon Hancock",
        "user_preferences": """
            Me gusta jugar Pickleball, Disc Golf y Tenis.
            Mi comida favorita es la mexicana.
            Mi programa de televisión favorito es Juego de Tronos.
            Le encanta cuando la gente le da "me gusta" y se suscribe a su canal de YouTube.
        """,
    }

    # Create a NEW session
    APP_NAME = "Brandon Bot"
    USER_ID = "brandon_hancock"
    SESSION_ID = str(uuid.uuid4())
    stateful_session = await session_service_stateful.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state=initial_state,
    )
    print("NUEVA SESIÓN CREADA:")
    print(f"\tID de Sesión: {SESSION_ID}")

    runner = Runner(
        agent=question_answering_agent,
        app_name=APP_NAME,
        session_service=session_service_stateful,
    )

    new_message = types.Content(
        role="user", parts=[types.Part(text="¿Cuál es el programa de televisión favorito de Brandon?")]
    )

    for event in runner.run(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=new_message,
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                print(f"Respuesta Final: {event.content.parts[0].text}")

    print("==== Exploración de Eventos de Sesión ====")
    session = await session_service_stateful.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
    )

    # Log final Session state
    print("=== Estado Final de la Sesión ===")
    for key, value in session.state.items():
        print(f"{key}: {value}")

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

Overwriting 26_ADK/5-sessions-and-state/basic_stateful_session.py


### Persistence Storage

In [13]:
import os
os.makedirs("26_ADK/6-persistent-storage/memory_agent", exist_ok=True)

In [15]:
%%writefile "26_ADK/6-persistent-storage/memory_agent/agent.py"
from google.adk.agents import Agent
from google.adk.tools.tool_context import ToolContext


def add_reminder(reminder: str, tool_context: ToolContext) -> dict:
    """Añade un nuevo recordatorio a la lista de recordatorios del usuario.

    Args:
        reminder: El texto del recordatorio a añadir
        tool_context: Contexto para acceder y actualizar el estado de la sesión

    Returns:
        Un mensaje de confirmación
    """
    print(f"--- Herramienta: add_reminder llamada para '{reminder}' ---")

    # Get current reminders from state
    reminders = tool_context.state.get("reminders", [])

    # Add the new reminder
    reminders.append(reminder)

    # Update state with the new list of reminders
    tool_context.state["reminders"] = reminders

    return {
        "action": "add_reminder",
        "reminder": reminder,
        "message": f"Recordatorio añadido: {reminder}",
    }


def view_reminders(tool_context: ToolContext) -> dict:
    """Ver todos los recordatorios actuales.

    Args:
        tool_context: Contexto para acceder al estado de la sesión

    Returns:
        La lista de recordatorios
    """
    print("--- Herramienta: view_reminders llamada ---")

    # Get reminders from state
    reminders = tool_context.state.get("reminders", [])

    return {"action": "view_reminders", "reminders": reminders, "count": len(reminders)}


def update_reminder(index: int, updated_text: str, tool_context: ToolContext) -> dict:
    """Actualiza un recordatorio existente.

    Args:
        index: El índice basado en 1 del recordatorio a actualizar
        updated_text: El nuevo texto para el recordatorio
        tool_context: Contexto para acceder y actualizar el estado de la sesión

    Returns:
        Un mensaje de confirmación
    """
    print(
        f"--- Herramienta: update_reminder llamada para el índice {index} con '{updated_text}' ---"
    )

    # Get current reminders from state
    reminders = tool_context.state.get("reminders", [])

    # Check if the index is valid
    if not reminders or index < 1 or index > len(reminders):
        return {
            "action": "update_reminder",
            "status": "error",
            "message": f"No se pudo encontrar el recordatorio en la posición {index}. Actualmente hay {len(reminders)} recordatorios.",
        }

    # Update the reminder (adjusting for 0-based indices)
    old_reminder = reminders[index - 1]
    reminders[index - 1] = updated_text

    # Update state with the modified list
    tool_context.state["reminders"] = reminders

    return {
        "action": "update_reminder",
        "index": index,
        "old_text": old_reminder,
        "updated_text": updated_text,
        "message": f"Recordatorio {index} actualizado de '{old_reminder}' a '{updated_text}'",
    }


def delete_reminder(index: int, tool_context: ToolContext) -> dict:
    """Elimina un recordatorio.

    Args:
        index: El índice basado en 1 del recordatorio a eliminar
        tool_context: Contexto para acceder y actualizar el estado de la sesión

    Returns:
        Un mensaje de confirmación
    """
    print(f"--- Herramienta: delete_reminder llamada para el índice {index} ---")

    # Get current reminders from state
    reminders = tool_context.state.get("reminders", [])

    # Check if the index is valid
    if not reminders or index < 1 or index > len(reminders):
        return {
            "action": "delete_reminder",
            "status": "error",
            "message": f"No se pudo encontrar el recordatorio en la posición {index}. Actualmente hay {len(reminders)} recordatorios.",
        }

    # Remove the reminder (adjusting for 0-based indices)
    deleted_reminder = reminders.pop(index - 1)

    # Update state with the modified list
    tool_context.state["reminders"] = reminders

    return {
        "action": "delete_reminder",
        "index": index,
        "deleted_reminder": deleted_reminder,
        "message": f"Recordatorio {index} eliminado: '{deleted_reminder}'",
    }


def update_user_name(name: str, tool_context: ToolContext) -> dict:
    """Actualiza el nombre del usuario.

    Args:
        name: El nuevo nombre para el usuario
        tool_context: Contexto para acceder y actualizar el estado de la sesión

    Returns:
        Un mensaje de confirmación
    """
    print(f"--- Herramienta: update_user_name llamada con '{name}' ---")

    # Get current name from state
    old_name = tool_context.state.get("user_name", "")

    # Update the name in state
    tool_context.state["user_name"] = name

    return {
        "action": "update_user_name",
        "old_name": old_name,
        "new_name": name,
        "message": f"Tu nombre ha sido actualizado a: {name}",
    }


# Create a simple persistent agent
memory_agent = Agent(
    name="memory_agent",
    model="gemini-2.0-flash",
    description="Un agente de recordatorios inteligente con memoria persistente",
    instruction="""
    Eres un amigable asistente de recordatorios que recuerda a los usuarios a través de las conversaciones.
    
    La información del usuario se almacena en el estado:
    - Nombre del usuario: {user_name}
    - Recordatorios: {reminders}
    
    Puedes ayudar a los usuarios a gestionar sus recordatorios con las siguientes capacidades:
    1. Añadir nuevos recordatorios
    2. Ver recordatorios existentes
    3. Actualizar recordatorios
    4. Eliminar recordatorios
    5. Actualizar el nombre del usuario
    
    Sé siempre amigable y dirígete al usuario por su nombre. Si aún no conoces su nombre,
    usa la herramienta update_user_name para almacenarlo cuando se presenten.
    
    **DIRECTRICES DE GESTIÓN DE RECORDATORIOS:**
    
    Al tratar con recordatorios, debes ser inteligente para encontrar el recordatorio correcto:
    
    1. Cuando el usuario pide actualizar o eliminar un recordatorio pero no proporciona un índice:
       - Si mencionan el contenido del recordatorio (ej. "eliminar mi recordatorio de reunión"), 
         busca en los recordatorios para encontrar una coincidencia
       - Si encuentras una coincidencia exacta o cercana, usa ese índice
       - Nunca aclares a qué recordatorio se refiere el usuario, simplemente usa la primera coincidencia
       - Si no se encuentra ninguna coincidencia, lista todos los recordatorios y pide al usuario que especifique
    
    2. Cuando el usuario menciona un número o posición:
       - Usa eso como el índice (ej. "eliminar recordatorio 2" significa índice=2)
       - Recuerda que la indexación comienza en 1 para el usuario
    
    3. Para posiciones relativas:
       - Maneja "primero", "último", "segundo", etc. apropiadamente
       - "Primer recordatorio" = índice 1
       - "Último recordatorio" = el índice más alto
       - "Segundo recordatorio" = índice 2, y así sucesivamente
    
    4. Para ver:
       - Siempre usa la herramienta view_reminders cuando el usuario pida ver sus recordatorios
       - Formatea la respuesta en una lista numerada para mayor claridad
       - Si no hay recordatorios, sugiere añadir algunos
    
    5. Para añadir:
       - Extrae el texto real del recordatorio de la solicitud del usuario
       - Elimina frases como "añadir un recordatorio para" o "recuérdame que"
       - Concéntrate en la tarea en sí (ej. "añadir un recordatorio para comprar leche" → add_reminder("comprar leche"))
    
    6. Para actualizaciones:
       - Identifica tanto qué recordatorio actualizar como cuál debe ser el nuevo texto
       - Por ejemplo, "cambiar mi segundo recordatorio a recoger la compra" → update_reminder(2, "recoger la compra")
    
    7. Para eliminaciones:
       - Confirma la eliminación cuando esté completa y menciona qué recordatorio se eliminó
       - Por ejemplo, "He eliminado tu recordatorio para 'comprar leche'"
    
    Recuerda explicar que puedes recordar su información a través de las conversaciones.

    IMPORTANTE:
    - usa tu mejor juicio para determinar a qué recordatorio se refiere el usuario. 
    - No tienes que ser 100% correcto, pero intenta ser lo más cercano posible.
    - Nunca le pidas al usuario que aclare a qué recordatorio se refiere.
    """,
    tools=[
        add_reminder,
        view_reminders,
        update_reminder,
        delete_reminder,
        update_user_name,
    ],
)


Overwriting 26_ADK/6-persistent-storage/memory_agent/agent.py


In [20]:
%%writefile "26_ADK/6-persistent-storage/main.py"
import asyncio

from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from memory_agent.agent import memory_agent
from utils import call_agent_async

load_dotenv()

# ===== PARTE 1: Inicializar el Servicio de Sesión Persistente =====
# Usando la base de datos SQLite para almacenamiento persistente
db_url = "sqlite:///./my_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)


# ===== PARTE 2: Definir el Estado Inicial =====
# Esto solo se usará al crear una nueva sesión
initial_state = {
    "user_name": "Brandon Hancock",
    "reminders": [],
}


async def main_async():
    # Configurar constantes
    APP_NAME = "Agente de Memoria"
    USER_ID = "aiwithbrandon"

    # ===== PARTE 3: Gestión de Sesiones - Encontrar o Crear =====
    # Buscar sesiones existentes para este usuario
    existing_sessions = await session_service.list_sessions(
        app_name=APP_NAME,
        user_id=USER_ID,
    )

    # Si hay una sesión existente, usarla, de lo contrario crear una nueva
    if existing_sessions and len(existing_sessions.sessions) > 0:
        # Usar la sesión más reciente
        SESSION_ID = existing_sessions.sessions[0].id
        print(f"Continuando sesión existente: {SESSION_ID}")
    else:
        # Crear una nueva sesión con el estado inicial
        new_session = await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            state=initial_state,
        )
        SESSION_ID = new_session.id
        print(f"Nueva sesión creada: {SESSION_ID}")

    # ===== PARTE 4: Configuración del Ejecutor del Agente =====
    # Crear un ejecutor con el agente de memoria
    runner = Runner(
        agent=memory_agent,
        app_name=APP_NAME,
        session_service=session_service,
    )

    # ===== PARTE 5: Bucle de Conversación Interactiva =====
    print("\n¡Bienvenido al Chat del Agente de Memoria!")
    print("Tus recordatorios serán recordados en todas las conversaciones.")
    print("Escribe 'exit' o 'quit' para finalizar la conversación.\n")

    while True:
        # Obtener la entrada del usuario
        user_input = input("Tú: ")

        # Comprobar si el usuario quiere salir
        if user_input.lower() in ["exit", "quit"]:
            print("Finalizando conversación. Tus datos han sido guardados en la base de datos.")
            break

        # Procesar la consulta del usuario a través del agente
        await call_agent_async(runner, USER_ID, SESSION_ID, user_input)


if __name__ == "__main__":
    asyncio.run(main_async())


Overwriting 26_ADK/6-persistent-storage/main.py


### Multi-agent

ADK permite crear sub-agntes y agentes como tools. El Agente-como-Herramienta te permite invocar a otro agente para realizar una tarea específica, delegando efectivamente la responsabilidad. Esto es conceptualmente similar a crear una función Python que llama a otro agente y utiliza la respuesta del agente como el valor de retorno de la función.

Diferencia clave con los sub-agentes
Es importante distinguir un Agente-xºcomo-Herramienta de un Sub-Agente.

- Agente-como-Herramienta: Cuando el Agente A llama al Agente B como una herramienta (usando Agente-como-Herramienta), la respuesta del Agente B se devuelve al Agente A, que luego resume la respuesta y genera una contestación al usuario. El Agente A mantiene el control y continúa manejando futuras entradas del usuario.

- Sub-agente: Cuando el Agente A llama al Agente B como un sub-agente, la responsabilidad de responder al usuario se transfiere completamente al Agente B. El Agente A queda efectivamente fuera del circuito. Todas las entradas de usuario subsiguientes serán respondidas por el Agente B.


### Stateful Multi-agent

### Callbacks

### Sequential Agent

### Parallel Agent

### Integración de MCP